In [1]:
import torch

from helpers.model.util import select_device
from helpers.experiment import Experiment
from helpers.data.dset.config import Config_Dataset
from helpers.data.dset.constants import (
    Names_Datasets,
    Names_Levels,
    Names_q_Squared_Vetos,
    Names_Splits,
    Names_Variables,
    Nums_Events_Per_Set
)
from helpers.model.config import Config_Model
from helpers.model.constants import Names_Models


path_dir_plots = "../../state/new_physics/plots"
path_dir_dsets_main = "../../state/new_physics/data/processed"
path_dir_raw_signal = "../../state/new_physics/data/raw/signal"
path_dir_raw_bkg = "../../state/new_physics/data/raw/bkg"
path_dir_models_main = "../../state/new_physics/models"

device = select_device()

value_dc9_np = -0.82

experiment = Experiment(
    path_dir_plots=path_dir_plots,
    device=device,
)


Device:  cuda


Images Method

In [2]:

num_sets_per_label_nominal = 50
num_sets_per_label_sens = 2000

kwargs_dsets = dict(
    name=Names_Datasets().images,
    q_squared_veto=Names_q_Squared_Vetos().loose,
    balanced_classes=True,
    std_scale=True,
    shuffle=True,
    num_bins_image=10,
    frac_bkg=0.5,
    path_dir_dsets_main=path_dir_dsets_main,
    path_dir_raw_signal=path_dir_raw_signal,
    path_dir_raw_bkg=path_dir_raw_bkg,
)

configs_dsets = {

    level : {

        num_events_per_set : (
            
            dict(

                train = (
                    Config_Dataset(
                        level=level,
                        split=Names_Splits().train,
                        num_events_per_set=num_events_per_set,
                        num_sets_per_label=num_sets_per_label_nominal,
                        **kwargs_dsets,
                    )
                ),

                eval = (
                    Config_Dataset(
                        level=level,
                        split=Names_Splits().eval_,
                        num_events_per_set=num_events_per_set,
                        num_sets_per_label=num_sets_per_label_nominal,
                        **kwargs_dsets,
                    )
                ),

                eval_sens = (
                    Config_Dataset(
                        level=level,
                        split=Names_Splits().eval_,
                        num_events_per_set=num_events_per_set,
                        num_sets_per_label=num_sets_per_label_sens,
                        label_subset=[value_dc9_np],
                        sensitivity_study=True,
                        **kwargs_dsets,
                    )
                ),
            )
        )

        for num_events_per_set in Nums_Events_Per_Set().tuple_
    }

    for level in Names_Levels().tuple_
}

kwargs_models = dict(
    name=Names_Models().cnn,
    path_dir_models_main=path_dir_models_main,
    loss_fn=torch.nn.MSELoss(),
    learn_rate=4e-4,
    size_batch_train=32,
    size_batch_eval=32,
    num_epochs=80,
    num_epochs_checkpoint=5,
)

configs_models = {

    level : {

        num_events_per_set : (

            Config_Model(
                config_dset_train=configs_dsets[level][num_events_per_set]["train"],
                **kwargs_models,
            )

        )

        for num_events_per_set in Nums_Events_Per_Set().tuple_
    
    }

    for level in Names_Levels().tuple_

}


Train

In [ ]:

for level in Names_Levels().tuple_:

    for num_events_per_set in Nums_Events_Per_Set().tuple_:

        experiment.train(
            config_model=configs_models[level][num_events_per_set],
            config_dset_eval=configs_dsets[level][num_events_per_set]["eval"],
            generate_dsets=True,
        )

Eval

In [3]:
for level in Names_Levels().tuple_:

    for num_events_per_set in Nums_Events_Per_Set().tuple_:

        experiment.evaluate(
            config_model=configs_models[level][num_events_per_set], 
            config_dset_eval=configs_dsets[level][num_events_per_set]["eval"],
            config_dset_eval_sens=configs_dsets[level][num_events_per_set]["eval_sens"],
            generate_dsets=False, 
        )

Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\70000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:38: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\24000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:38: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_gen_q2v_loose\6000_eval_labels.pt
Loaded dataset: images


...

c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:38: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\24000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:38: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


Loaded tensor of shape: torch.Size([2000, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_sens_features.pt
Loaded tensor of shape: torch.Size([2000]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_sens_labels.pt
Loaded dataset: images
Loaded tensor of shape: torch.Size([2200, 1, 10, 10, 10]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_features.pt
Loaded tensor of shape: torch.Size([2200]) from: ..\..\state\new_physics\data\processed\images_det_bkg_q2v_loose\6000_eval_labels.pt
Loaded dataset: images


c:\Users\tetha\Desktop\btokstll\logic\scripts\helpers\result\table.py:38: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.table.loc[


In [4]:
experiment.table_summary.table

MSE       MAE Std. at NP  \
Level   q^2 Veto Method    Events / Set                                  
gen     tight    deep_sets 70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 cnn       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 ebe       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
        loose    deep_sets 70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 cnn       70000         0.029379  0.136679   0.148001   
                           24000         0.071332  0.207109   0.269582   
                           6000          0.254925  0.395323     0.4993   
                 ebe       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
det     tight    deep_sets 70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 cnn       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 ebe       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
        loose    deep_sets 70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 cnn       70000         0.040395  0.163231   0.153337   
                           24000         0.098551  0.243739   0.277648   
                           6000          0.381247  0.484002   0.559959   
                 ebe       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
det_bkg tight    deep_sets 70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 cnn       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 ebe       70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
        loose    deep_sets 70000              NaN       NaN        NaN   
                           24000              NaN       NaN        NaN   
                           6000               NaN       NaN        NaN   
                 cnn       70000         0.085421  0.240948   0.192714   
                           24000         0.197464  0.345673   0.349804   
                           6000          0.531169  0.575558   0.620165   
                 ebe       70000              NaN       NaN        NaN   
                           24000              NaN 

In [13]:
from math import sqrt
import pathlib

import matplotlib.pyplot as plt
import pandas

from helpers.plot.util import add_plot_note
from helpers.result.constants import Names_Kinds_Items


def plot(table):

    y_lims = [
        (0, 0.55), 
        (0, 0.6), 
        (0, 0.65), 
        (-1.5, 0.5), 
        (-0.5, 0.5)
    ]

    for col, y_lim in zip(
        table.columns, 
        y_lims
    ):

        _, ax = plt.subplots()
        
        for level, fmt, color in zip(
            Names_Levels().tuple_, 
            ["-o", "-v", "-s"],
            ["#999999", "#377eb8", "#a65628"],
        ):

            y = table.loc[
                pandas.IndexSlice[
                    level, 
                    kwargs_dsets["q_squared_veto"], 
                    kwargs_models["name"],
                ], 
                col,
            ]

            x = y.index

            ax.plot(
                x, 
                y, 
                fmt, 
                label=f"{kwargs_models["name"]}, {level}", 
                c=color, 
                markersize=5, 
                alpha=.8
                )
            
            if (
                (col==Names_Kinds_Items().np_bias) 
                or (col==Names_Kinds_Items().np_mean)
            ):
            
                errors = (
                    table.loc[
                        pandas.IndexSlice[
                            level, 
                            kwargs_dsets["q_squared_veto"], 
                            kwargs_models["name"],
                        ], 
                        Names_Kinds_Items().np_std,
                    ] 
                    / sqrt(num_sets_per_label_sens)
                )
            
                ax.errorbar(
                    x=x, 
                    y=y, 
                    yerr=errors, 
                    fmt='none', 
                    elinewidth=0.5, 
                    capsize=0.5, 
                    color="black",
                )

            ax.set_ylim(y_lim)
            ax.set_ylabel(f"{col}")
            ax.set_xlabel("Number of events / set")
            ax.legend(ncols=2, markerscale=0.5, numpoints=1)

            if (
                (col==Names_Kinds_Items().np_bias) 
                or (col==Names_Kinds_Items().np_mean) 
                or (col==Names_Kinds_Items().np_std)
            ):
                add_plot_note(
                    ax, 
                    f"Num boots.: {num_sets_per_label_sens}",
                )
            
            else: 
                add_plot_note(
                    ax, 
                    f"Num boots./label: {num_sets_per_label_nominal}",
                )

        file_name = f"comp_{kwargs_models["name"]}_{col}.png"
        file_path = pathlib.Path(path_dir_plots).joinpath(file_name)

        plt.savefig(file_path, bbox_inches="tight")
        plt.close()

plot(experiment.table_summary.table)

C:\Users\tetha\AppData\Local\Temp\ipykernel_1200\2714503057.py:34: PerformanceWarning: indexing past lexsort depth may impact performance.
  y = table.loc[
C:\Users\tetha\AppData\Local\Temp\ipykernel_1200\2714503057.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.
  table.loc[


In [ ]:
import pandas

df_charge = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_whole.pkl")

df_mix = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_whole.pkl")


In [ ]:
def split(df):
    ind_half = int(len(df)/2)
    df_train = df[:ind_half].copy()
    df_eval = df[ind_half:].copy()
    return df_train, df_eval

df_charge_train, df_charge_eval = split(df_charge.loc["det"][list(Names_Variables().tuple_)])

df_mix_train, df_mix_eval = split(df_mix.loc["det"][list(Names_Variables().tuple_)])


In [ ]:
pandas.to_pickle(df_charge_train, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")

In [ ]:
pandas.to_pickle(df_charge_eval, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")


In [ ]:
pandas.to_pickle(df_mix_train, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")


In [ ]:
pandas.to_pickle(df_mix_eval, "../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_eval.pkl")


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")

In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_eval.pkl")


In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")

In [ ]:
pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_eval.pkl")

In [ ]:
import pandas
from helpers.data.dset.preproc import apply_q_squared_veto


df_bkg_charge = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_charge_train.pkl")
df_bkg_mix = pandas.read_pickle("../../state/new_physics/data/raw/bkg/mu_sideb_generic_mix_train.pkl")

df_bkg = pandas.concat([df_bkg_charge, df_bkg_mix])

df_agg = pandas.read_pickle("../../state/new_physics/data/processed/agg_sig_1_to_20_det.pkl")

df_combo = pandas.concat([df_bkg[list(Names_Variables().tuple_)], df_agg[list(Names_Variables().tuple_)]])

df_combo_q2v_tight = apply_q_squared_veto(df_combo, "tight")

df_combo_q2v_tight.mean()



In [ ]:
df_agg